In [1]:
! wget https://storage.googleapis.com/kaggle-forum-message-attachments/237294/7771/german_credit_data.csv

--2018-11-23 21:52:56--  https://storage.googleapis.com/kaggle-forum-message-attachments/237294/7771/german_credit_data.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.13.112, 2607:f8b0:4020:804::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.13.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53393 (52K) [text/csv]
Saving to: ‘german_credit_data.csv’

german_credit_data. 100%[===================>]  52.14K  --.-KB/s    in 0.03s   

2018-11-23 21:52:56 (1.61 MB/s) - ‘german_credit_data.csv’ saved [53393/53393]



In [4]:
import pandas as pd
df = pd.read_csv("german_credit_data.csv", index_col=0)

In [5]:
df = df.fillna(value="NA")

In [6]:
df

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NA,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NA,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
5,35,male,1,free,NA,NA,9055,36,education,good
6,53,male,2,own,quite rich,NA,2835,24,furniture/equipment,good
7,35,male,3,rent,little,moderate,6948,36,car,good
8,61,male,1,own,rich,NA,3059,12,radio/TV,good
9,28,male,3,own,little,moderate,5234,30,car,bad


In [7]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
preprocess = make_column_transformer(
    (['Age', 'Credit amount', 'Duration'], StandardScaler()),
    (['Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account', 'Purpose', 'Risk'], OneHotEncoder(sparse=False))
)

/home/ashudeep/anaconda3/lib/python3.6/site-packages/sklearn/compose/_column_transformer.py:739: DeprecationWarning: `make_column_transformer` now expects (transformer, columns) as input tuples instead of (columns, transformer). This has been introduced in v0.20.1. `make_column_transformer` will stop accepting the deprecated (columns, transformer) order in v0.22.
  warnings.warn(message, DeprecationWarning)


In [8]:
mat = preprocess.fit_transform(df)

/home/ashudeep/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ashudeep/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [9]:
mat[10,:]

array([-0.92754658, -0.70047155, -0.73866754,  1.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ])

In [10]:
X = mat[:, :-2]
Y = mat[:, -1]

In [11]:
num_feats = X.shape[1]
numX = X.shape[0]

In [12]:
import numpy as np

In [13]:
datasize = 500
cs_size = 25
split_on_doc = 0.8
testsize = 100
ratio_relevant = 0.4
ratios_col = Y * ratio_relevant + (1-Y)*(1-ratio_relevant)

In [14]:
# generate a candidate set of size 10 everytime
data_X = []
data_Y = []
test_X = []
test_Y = []
group_identities_train = []
group_identities_test = []
print("Sampling between 0 and {} for train".format(numX*split_on_doc))
p = ratios_col[0:int(numX*split_on_doc)]
p = p / sum(p)
for i in range(datasize):
    cs_indices = np.random.choice(np.arange(0, numX*split_on_doc, dtype=int), size=cs_size, p=p)
    cs_X = X[cs_indices]
    cs_Y = Y[cs_indices]
    data_X.append(cs_X)
    data_Y.append(cs_Y)
    group_identities_train.append(cs_X[:,4])
print("Sampling between {} and {} for test".format(numX*split_on_doc, numX))
p = ratios_col[int(numX*split_on_doc):]
p = p/sum(p)
for i in range(testsize):
    cs_indices = np.random.choice(np.arange(numX*split_on_doc, numX, dtype=int), size=cs_size, p=p)
    test_X.append(X[cs_indices])
    test_Y.append(Y[cs_indices])
    group_identities_test.append(X[cs_indices,4])

Sampling between 0 and 800.0 for train
Sampling between 800.0 and 1000 for test


In [15]:
import pickle as pkl
pkl.dump((data_X, data_Y), open("german_train_rank_3.pkl", "wb"))
pkl.dump((test_X, test_Y), open("german_test_rank_3.pkl", "wb"))
